## 9.4 パラメータの制約

- 9.4.1 ･･･ lowerとupperによる範囲制限の機能
- 9.4.2 ･･･ simplex型
- 9.4.3 ･･･ ordered型
- 9.4.4 ･･･ その他の制約（transformed parametersブロックの利用例）

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import stan_jupyter as stan
import arviz as az
from PIL import Image
import numpy as np

from utils.glaph import pairplot

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

### 9.4.1 lowerとupperによる範囲制限

In [ ]:
stan_parameters_sample = """
parameters {
    real a[N, K]
    real<lower=a[1, 1], upper=a[2, 2]> b;    # 事前に宣言された変数を使うことができる
    real<lower=min(a[1])> c;                 # Stanで用意されている関数を使うこともできる
    vector[K] beta1[N];
    vector<lower=-10, upper=10>[K] beta2;    # vector型の要素に制限をかける場合は指定位置に注意. vector[K]<lower･･･>ではない
    vector<lower=min(beta1[2])>[K] beta3;    # beta1[2]が（N個中2個目の）長さKのvector型を返すのでその中のminを制限に使うことができる
}
"""

### 9.4.2 simplex型

- simplex型はvector型の特別な場合
- 各要素が[0, 1]の範囲で合計が1という条件を満たす
- 確率とみなせるのでカテゴリカル分布や多項分布のパラメータとして使うことができる

例

In [3]:
# 6面サイコロを200回振って出た目のデータ
df_dice = pd.read_csv("./data/data-dice.txt")
df_dice.head()

,Face
0,1
1,2
2,6
3,5
4,4


#### モデル式9-4

$$
Y[n] \sim Categorical(\vec{\theta}) \quad \quad n = 1, \dots, N
$$

- $N$ : 試行回数（$N=200$）
- $n$ : サイコロのインデックス
- $Y$ : 出た目
- $\vec{\theta}$ : 長さKのベクトル（$K=6$）であり、サイコロの各目が出る確率を格納する. (ex) [0.16, 0.16, 0.17, 0.17, 0.17, 0.17]
- $\vec{\theta}$ の事前分布は無情報事前分布とする  
    → ただし、サイコロを作るプロセスを解析者が知っており、どのような形のサイコロができやすいか知っている場合には弱情報事前分布を設定すると良い（10.2.3項）

In [7]:
stan_model = """
data {
    int N;
    int K;
    int<lower=1, upper=K> Y[N];
}

parameters {
    simplex[K] theta;
}

model {
    for (n in 1:N) {
        Y[n] ~ categorical(theta);
    }
}
"""

stan_data = {
    "N": 200,
    "K": 6,
    "Y": df_dice["Face"].values.astype(int),
}

posterior = stan.build(stan_model, data=stan_data, random_seed=123)
fit = posterior.sample()
fit.to_frame().describe()

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_rxjney4j/model_k76rl56c.stan', line 5, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 0.000192 seconds
  1000 transitions using 10 leapfrog steps per transition would take 1.92 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5.4e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.54 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 3.8e-05 seconds
  1000 transitions using 10 leapfrog steps per transition

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-332.999560,0.898003,0.776968,2.510500,5.39300,0.0,335.504990,0.106810,0.368501,0.101909,0.253072,0.096877,0.072831
std,1.608981,0.112877,0.040290,0.509858,1.96431,0.0,2.309427,0.021734,0.033727,0.021283,0.030148,0.020157,0.018197
min,-343.978833,0.221976,0.736556,1.000000,1.00000,0.0,330.876632,0.045854,0.227612,0.039817,0.152353,0.037600,0.025813
25%,-333.807219,0.845030,0.736746,2.000000,3.00000,0.0,333.834037,0.091011,0.344976,0.086755,0.232579,0.082492,0.059899
50%,-332.693195,0.936599,0.776982,3.000000,7.00000,0.0,335.135435,0.105554,0.368013,0.100893,0.252363,0.095585,0.071179
75%,-331.826869,0.987906,0.817203,3.000000,7.00000,0.0,336.724575,0.120894,0.391463,0.115441,0.272093,0.109720,0.084559
max,-330.520353,1.000000,0.817351,3.000000,7.00000,0.0,349.323405,0.192903,0.493105,0.184675,0.353291,0.200928,0.142181


※1回ごとのサイコロの出た目を扱うよりも、出た目の集計を取ってから多項分布を使った方がはるかに高速

#### モデル式9-5

$$
\vec{Y} \sim Multinormial(N, \vec{\theta})
$$

- $\vec{Y}$ は長さKのベクトルで、$Y[1]$ はN海中１の目が出た回数、$Y[2]$ は2の目が出た回数、...となっている
- Yの各要素は各目の出た回数の集計値（int型）のため、vector型で宣言できない

In [16]:
stan_model = """
data {
    int K;
    int<lower=0> Y[K];
}

parameters {
    simplex[K] theta;
}

model {
    Y ~ multinomial(theta);
}
"""

stan_data = {
    "K": 6,
    "Y": df_dice["Face"].value_counts().sort_index().tolist(),
}

posterior = stan.build(stan_model, data=stan_data, random_seed=123)
fit = posterior.sample()
fit.to_frame().describe()

Building...



Building: found in cache, done.Messages from stanc:
Warning in '/tmp/httpstan_hnvf0pq_/model_gvvpxqy6.stan', line 4, column 4: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Sampling:   0%
Sampling: 100%, done.
Messages received during sampling:
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 7e-06 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.07 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 5e-06 seconds
  1000 transitions u

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,theta.1,theta.2,theta.3,theta.4,theta.5,theta.6
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.0,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,-332.970447,0.906313,0.722251,2.575000,5.595500,0.0,335.474906,0.106896,0.369195,0.102419,0.251344,0.097508,0.072638
std,1.599790,0.102931,0.024448,0.505904,1.912405,0.0,2.290357,0.021656,0.034103,0.020672,0.029992,0.020062,0.018180
min,-342.729535,0.336841,0.700718,1.000000,1.000000,0.0,330.914525,0.043214,0.258475,0.045769,0.140954,0.039693,0.023694
25%,-333.781235,0.853920,0.706922,2.000000,3.000000,0.0,333.830488,0.091435,0.346885,0.088171,0.230484,0.083920,0.059246
50%,-332.664857,0.941968,0.712353,3.000000,7.000000,0.0,335.155219,0.105682,0.368736,0.101379,0.250855,0.096392,0.071500
75%,-331.772780,0.989800,0.727682,3.000000,7.000000,0.0,336.698868,0.120931,0.391995,0.115601,0.271194,0.109900,0.084055
max,-330.521009,1.000000,0.763579,3.000000,7.000000,0.0,352.076773,0.191442,0.523179,0.201424,0.372510,0.185242,0.147570


### 9.4.3 orderd型

- 順序付きベクトル（ordered vector）
- 長さNのordered型は要素が $x_1 \lt x_2 \lt \cdots \lt x_N$ を満たす

例

- ある雑誌において、どの連載が一番面白いか帳票したデータがあるとする
- この順序付きのデータから、それぞれの連載の「真の面白さ」を大小関係があるパラメータと考えてordered型で宣言して推定する
- 10.1.5項、10.2.2項に類似した例がある

- 識別性（10.1節参照）を入れるために使う。11.2節に例がある

- 正の値のみの場合は positive_ordered型を使う
- ordered型でlowerやupperを指定するとエラーになる

### 9.4.4 その他の制約

- Stanでパラメータの値を制限するには9.4.1~9.4.3の方法が効率的
- それでできない制約については、transformed parametersブロックにおいて変数変換して実現する必要がある

#### 例：各要素が[0, 1] の範囲で大小関係があるパラメータ（$p_1 \gt p_2 \gt \dots$）

- inv_logitで[0, 1]の範囲にする
- ordered型の制限によって要素の値が小さい順になるので添え字を逆にする


In [29]:
sample = """
※dataは省略

parameters {
    ordered[N] p_x;
}

transformed parameters {
    vector[N] p;
    for (n in 1:N) {
        p[n] = inv_logit(p_x[N-n+1]);
    }
}
"""

In [28]:
def inv_logit(z: np.ndarray) -> np.ndarray:
    """ロジスティック関数"""
    return 1 / (1 + np.exp(-z))

v = np.array([-3, -2, -1, 1, 2, 3])

# 正の値にする
print(inv_logit(v))
# 大きい順にする
print(inv_logit(v[::-1]))

[0.04742587 0.11920292 0.26894142 0.73105858 0.88079708 0.95257413]
[0.95257413 0.88079708 0.73105858 0.26894142 0.11920292 0.04742587]


#### 例：切断正規分布や半ｔ分布（切断されたような分布の場合）

- upperやlowerで範囲制限を加えるだけで実装できる

ただし、切断正規分布の標準偏差や切断点がパラメータの場合には、正規化項が定数にならないので、その分の対数確率を加える必要がある。（興味のある人はStanマニュアル参照とのこと）

In [ ]:
sample = """
parameters {
    real<lower=-1, upper=3> a;
    real<lower=0> b;
}

model {
    a ~ normal(0, 1);
    b ~ student_t(4, 0, 1);    # student_t(nu, mu, sigma)
}
"""

#### 例：等式を満たすパラメータ

長さNのベクトルで各要素が $(-\infty, \infty)$ の範囲で合計が12.3になるパラメータを定義した場合

In [32]:
sample = """
parameters {
    vector[N-1] x0;
}

transformed parameters {
    vector[N] x;
    x[1:(N-1)] = x0;
    x[N] = 12.3 - sum(x0);
}
"""

#### 例：不等式を満たすパラメータ（その1）

先ほどの例で、合計を「12.3以上」としたい場合

In [ ]:
sample = """
parameters {
    vector[N-1] x0;
    real<lower=12.3-sum(x0)> xN;
}

transformed parameters {
    vector[N] x;
    x[1:(N-1)] = x0;
    x[N] = xN;
}
"""

#### 例：不等式を満たすパラメータ（その2）

二つのパラメータa, bが $a^2 + b^2 \gt 1$ を満たすように制限したい場合

次の書き方は非推奨  
    →transformed parametersで制限をかけるとサンプリング効率が著しく下がるため、デバッグ目的でなければ推奨されない

In [ ]:
sample = """
parameters {
    real a;
    real b;
}

transformed parameters {
    real<lower=1> theta;
    theta = square(a) + square(b);
}

model {
    a ~ normal(0, 1);
    b ~ normal(0, 1);
}
"""

以下はBox-Muller変換の例

In [ ]:
sample = """
parameters {
    real<lower=0, upper=2*pi()> theta;
    real<lower=0, upper=exp(-0.5)> U1;
}

transformed parameters {
    real R;
    real a;
    real b;
    R = sqrt(-2 * log(U1))
    a = R * cos(theta);
    b = R * sin(theta);
}

model {
    a ~ normal(0, 1);
    b ~ normal(0, 1);
}
"""

## 9.5 トラブルシューティング

### 9.5.1 int型のパラメータ（Stanの最大の弱点）

- parametersブロック内において、int a や int<lower=1> b[N] を使用できない
- vector型だと実数値になりint型のパラメータに使えない
- Stanのサンプリングアルゴリズム（NUTS）がHMCの一種であり、整数値のような離散的な値を取るパラメータを扱う理論がまだ存在しないため

→この対処法は11章で解説

### 9.5.2 欠損値

- stanではNAはエラーになる
- 下記例ではPersonID：2や3の行が使えない（Time4など、値がある項目もあるのに･･･）
- 横長形式でデータを持つより縦長形式でデータを持つ方が良い

In [33]:
df_wide = pd.read_csv("./data/data-conc-2-NA-wide.txt")
df_wide.head(10)

,PersonID,Time1,Time2,Time4,Time8,Time12,Time24
0,1,2.4,NaN,7.5,11.9,12.5,NaN
1,2,NaN,3.9,4.4,7.7,6.4,8.3
2,3,5.2,9.4,19.4,NaN,NaN,NaN
3,4,6.7,12.6,19.1,23.4,25.8,26.1
4,5,0.3,4.7,7.0,10.2,12.9,14.8
5,6,6.3,3.8,11.8,9.2,13.9,18.2
6,7,3.0,4.2,8.8,15.4,10.7,16.2
7,8,6.2,6.8,9.4,11.3,12.4,14.7
8,9,14.4,17.0,22.7,29.8,33.0,32.2
9,10,7.7,10.0,14.8,15.3,18.0,18.7


In [34]:
df_long = pd.read_csv("./data/data-conc-2-NA-long.txt")
df_long.head(10)

,PersonID,TimeID,Y
0,1,1,2.4
1,3,1,5.2
2,4,1,6.7
3,5,1,0.3
4,6,1,6.3
5,7,1,3.0
6,8,1,6.2
7,9,1,14.4
8,10,1,7.7
9,11,1,8.8


#### モデル式9-6（モデル式8-7 P139 のデータの持ち方を縦長に変更）

$$
\begin{align}
  Y[i] &\sim Normal(\mu[PersonID[i], TimeID[i]], \sigma_Y) \qquad i=1,\dots,I \\
  \mu[n, t] &= a[n]\{ 1- exp(-b[n] Time[t]) \} \qquad \qquad \qquad \quad \ n=1,\dots,N \quad t=1,\dots,T\\
  \log{(a[n])} &\sim Normal(a_{全体平均}, \sigma_a) \qquad \qquad \qquad \qquad \qquad \ \ n=1,\dots,N \\
  \log{(b[n])} &\sim Normal(b_{全体平均}, \sigma_b) \qquad \qquad \qquad \qquad \qquad \ \ \ n=1,\dots,N \\
\end{align}
$$


再掲（モデル式8-7）

$$
\begin{align}
  Y[n,t] &\sim Normal(a[n] \{ 1-exp(-b[n] Time[t]) \}, \sigma_Y) \quad n=1,\dots,N \quad t=1,\dots,T\\
  \log{(a[n])} &\sim Normal(a_{全体平均}, \sigma_a) \qquad \qquad \qquad \qquad \qquad \ \ n=1,\dots,N \\
  \log{(b[n])} &\sim Normal(b_{全体平均}, \sigma_b) \qquad \qquad \qquad \qquad \qquad \ \ \ n=1,\dots,N \\
\end{align}
$$

In [ ]:
sample = """
data {
  int I;
  int N;
  int T;
  real Time[T];
  int<lower=1, upper=N> PersonID[I];
  int<lower=1, upper=T> TimeID[I];
  vector[I] Y;
}

parameters {
  real a0;
  real b0;
  vector[N] log_a;
  vector[N] log_b;
  real<lower=0> s_a;
  real<lower=0> s_b;
  real<lower=0> s_Y;
}

transformed parameters {
  vector[N] a;
  vector[N] b;
  row_vector[T] mu[N];
  a = exp(log_a);
  b = exp(log_b);
  for (n in 1:N)
    for (t in 1:T)
      mu[n,t] = a[n]*(1 - exp(-b[n]*Time[t]));
}

model {
  log_a ~ normal(a0, s_a);
  log_b ~ normal(b0, s_b);
  for (i in 1:I)
    Y[i] ~ normal(mu[PersonID[i], TimeID[i]], s_Y);
}
"""

### 9.5.3 Stanのエラーメッセージ

#### コンパイル時



##### ■「;」を忘れた
```
PARSER EXPECTED: ";"  
```  
が表示される

===============================================================================

##### ■ 宣言していない変数を使用した  
```
variable "y" does not exist.  
```  
が表示される


transformed parametersブロックで宣言されていない変数zを使用した場合
```
unknown variable in assignment; lhs variable=z  
```  
が表示される

===============================================================================

##### ■ int型のパラメータを宣言した  

```
integer parameters or transformed parameters are not allowed; found declared type int,
```  
が表示される

===============================================================================

##### ■ vectorの長さを記述する位置を間違えた  

```
PARSER EXPECTED: "["     
```
が表示される

===============================================================================

##### ■ 関数の引数が求める型と一致しない
  
例えば、$Y \sim binomial(N, p)$ とすべきところ　$Y \sim binomial(p, N)$ とした場合  

```
No matches for:
    int ~ binomial(real, int)
```

===============================================================================

##### ■ modelブロックにおいて、parametersブロックで宣言されたパラメータを「=（代入）」で定義した

```
attempt to assign variable in wrong block.
```

In [38]:
# 例
_ = """
model {
    mu = a + b*X
    Y ~ normal(mu, sigma)
}
"""

# 上記のようにしたい場合、transformed parametersブロックに記述する

_ = """
transformed parameters {
    mu = a + b*X
}

model {
    Y ~ normal(mu, sigma)
}
"""

===============================================================================

##### ■ int型をint型で割る

```
Warning: integer division implicitly rounds to integer. Found int division: sum(Y) / N
```

In [41]:
# 例
_ = """
data {
    int N;     # 例: 10とする
    int Y[N];  # 例: 53とする
}

transformed parameters {
    mu = sum(Y) / N    # 53 / 10 = 商5余り3 となる
}
"""

# 次のようにするとエラーにならなくなる（int型がreal型に変換される）

_ = """
transformed parameters {
    mu = 1.0*sum(Y) / N    # 53.0 / 10 = 5.3 となる
}
"""

===============================================================================

#### 実行時

##### ■ モデルの初期値が適切でないため、サンプリングが進まない  

次の例の場合、sigmaの初期値次のようになる。（4.4.9項参照）  

$$
x \sim uniform(-2, 2) \\
sigma_{init} = 100/(1+exp(-x))
$$

生成された初期値が[0, 1]の範囲外であるためエラーになる

In [42]:
_ = """
parameters {
    real<lower=0, upper=100> sigma;
}

model {
    sigma ~ uniform(0, 1)
}
"""

同様に次の場合

$Y \sim bernoulli(inv\_logint(\mu-100))$

初期値を指定しないと $\mu$ は[-2, 2]の範囲で初期化されるため、$inv\_logint(\mu-100) \approx 0$ となりエラーとなる。

$\mu$ の初期値を100近くに指定すればよい

===============================================================================

##### ■ Rからデータを渡すときに名前付きlistで渡していない

Rで list(N=nrow(d), K=6, Y=d$Y) のような名前付きリストでstan関数やsampling関数に渡さないとエラーとなる

===============================================================================

##### ■ サンプリングに関する警告

初期値付近ではサンプリングが失敗することも多いため、countの値が小さい場合は無視してよい

adapt_deltaの値の提案がある場合がある。指定方法は次の通り。（adapt_deltaについての詳細はNUTSの原論文を参照とのこと）

In [ ]:
_ = """
stan(file="model.stan", data=data, seed=1, control=list(adapt_delta=0.99))
"""

===============================================================================

### 9.5.4 print関数を使ったデバッグ

Stanでは任意のブロックでprint関数で変数の内容を出力することができる。（書き方はpythonに似ている･･･）

エラー時の原因調査に使える。

In [ ]:
print("y=", y, " z=", z);